In [1]:
!pip install "git+https://github.com/rawkintrevo/gofannon.git@19#egg=gofannon[headless_browser]" --quiet

DEPRECATION: git+https://github.com/rawkintrevo/gofannon.git@19#egg=gofannon[headless_browser] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 22.0 MB/s eta 0:00:00


## Show that a site can't be loaded with `requests.get`

The site https://app-template-37928.web.app is a failed attempt at teaching myself material in I want to say the late 2010s. It's a really small site, and if java script isn't enabled it will fail. Click and go to the site, you'll see it is very little besides 'home works!'. We try to load it with requests and don't even get that much.

In [2]:
from requests import get

r = get('https://app-template-37928.web.app/')

print('home works' in r.text)


False


## Using `headless_browser_get` we do see it.

In the next couple of blocks we'll load the same site with a headless browser, so the java script will render, and then we'll have the LLM describe the page to us.

In [3]:
from openai import OpenAI
from google.colab import userdata
import json
# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=userdata.get('deepinfra'),
    base_url="https://api.deepinfra.com/v1/openai",
)

from gofannon.headless_browser.headless_browser_get import HeadlessBrowserGet



tool_list =  [F() for F in [HeadlessBrowserGet]]
tool_map = {f.name: f.fn for f in tool_list}
tool_desc_list = [f.definition for f in tool_list]

/usr/local/lib/python3.11/dist-packages/gofannon/headless_browser/base.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ..base import BaseTool


In [4]:
# here is the user request
messages = [
    {
        "role": "user",
        "content": "Describe the text you find on the page https://app-template-37928.web.app"
    }
]

# let's send the request and print the response
response = openai.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
    messages=messages,
    tools=tool_desc_list,
    tool_choice="auto",
)
tool_calls = response.choices[0].message.tool_calls
for tool_call in tool_calls:
    print(tool_call.model_dump())

{'id': 'call_yZXQ93eKcBG3VcqgEn1LVizw', 'function': {'arguments': '{"url": "https://app-template-37928.web.app"}', 'name': 'headless_browser_get'}, 'type': 'function'}


In [5]:
messages.append(response.choices[0].message)

for tool_call in tool_calls:
  function_name = tool_call.function.name
  function_args = json.loads(tool_call.function.arguments)
  function_response = tool_map[function_name](**function_args)

  # extend conversation with function response
  messages.append({
      "tool_call_id": tool_call.id,
      "role": "tool",
      "content": function_response,
  })


# get a new response from the model where it can see the function responses
second_response = openai.chat.completions.create(
  model="meta-llama/Meta-Llama-3.1-70B-Instruct",
  messages=messages,
  tools=tool_desc_list,
  tool_choice="auto",
)

print(second_response.choices[0].message.content)

The text on the page https://app-template-37928.web.app is:

"home works!"

This is the content of the <h1> tag in the <app-home> component, which is rendered inside the <router-outlet> element in the main app component.

There is also a navigation menu with several links, including "update", "schedule", "face", and "login". The "login" link is a router link that navigates to the "/login" route.

Additionally, there is a toolbar with a toggle button for the side navigation menu, and a title that says "[AppTitle Here]".


## It Works

As we can see the headless_browser_get did a bang up job of fetching content that requires client side js rendering.